In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import xgboost as xgb
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA

In [2]:
train_df = pd.read_csv('train.csv')
train_df.sample(5)

,uid,NAME_CONTRACT_TYPE,TARGET,n_loans,total_loan_amount,mean_loan_amount,min_loan_amount,max_loan_amount,amount_overdue,overdue_ratio,...,late_days,late_months,loan_frequency,days_since_last_loan,n_enquiries,total_enquiry_amount,mean_enquiry_amount,min_enquiry_amount,max_enquiry_amount,enquiry_frequency
135507,SAZ48669448,Cash loans,0,10,1385941.68,138594.168,0.00,490500.0,0.0,0.0,...,0,0,0.004179,0,11,1369000,124454.545455,13000,199000,0.016616
213502,DTH33272394,Cash loans,0,9,17441662.50,1937962.500,15655.50,11371500.0,0.0,0.0,...,31,2,0.004386,0,18,1687000,93722.222222,6000,152000,0.023438
94846,UVO12568704,Cash loans,0,6,3324820.50,554136.750,0.00,1350000.0,0.0,0.0,...,0,0,0.007084,0,5,251000,50200.000000,27000,109000,0.012594
126811,PMH38262938,Cash loans,0,10,2191940.55,219194.055,31765.05,676800.0,0.0,0.0,...,9,1,0.005838,0,7,783000,111857.142857,24000,198000,0.008717
197650,ZQE48151025,Cash loans,0,2,2209860.00,1104930.000,196065.00,2013795.0,0.0,0.0,...,0,0,0.002571,0,1,73000,73000.000000,73000,73000,0.001000


In [3]:
train_df['NAME_CONTRACT_TYPE'] = train_df['NAME_CONTRACT_TYPE'].map({'Cash loans': 0, 'Revolving loans': 1})

In [4]:
X = train_df.loc[:, (train_df.columns != 'TARGET') & (train_df.columns != 'uid')]
y = train_df['TARGET']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=94)

# Use "hist" for constructing the trees, with early stopping enabled.
clf = xgb.XGBClassifier(scale_pos_weight=10)
# Fit the model, test sets are used for early stopping.
clf.fit(X_train, y_train, eval_set=[(X_test, y_test)])
# Save model into JSON format.
clf.save_model("clf.json")

[0]	validation_0-logloss:0.60941
[1]	validation_0-logloss:0.60138
[2]	validation_0-logloss:0.59629
[3]	validation_0-logloss:0.59211
[4]	validation_0-logloss:0.58937
[5]	validation_0-logloss:0.58688
[6]	validation_0-logloss:0.58465
[7]	validation_0-logloss:0.58276
[8]	validation_0-logloss:0.58002
[9]	validation_0-logloss:0.57820
[10]	validation_0-logloss:0.57733
[11]	validation_0-logloss:0.57620
[12]	validation_0-logloss:0.57469
[13]	validation_0-logloss:0.57305
[14]	validation_0-logloss:0.57208
[15]	validation_0-logloss:0.57107
[16]	validation_0-logloss:0.57025
[17]	validation_0-logloss:0.56910
[18]	validation_0-logloss:0.56867
[19]	validation_0-logloss:0.56797
[20]	validation_0-logloss:0.56736
[21]	validation_0-logloss:0.56660
[22]	validation_0-logloss:0.56609
[23]	validation_0-logloss:0.56581
[24]	validation_0-logloss:0.56503
[25]	validation_0-logloss:0.56443
[26]	validation_0-logloss:0.56410
[27]	validation_0-logloss:0.56347
[28]	validation_0-logloss:0.56254
[29]	validation_0-loglos

In [6]:
clf.score(X_test, y_test)

0.7480886030725259

In [7]:
y_pred = clf.predict(X_test)

In [8]:
confusion_matrix(y_test, y_pred)

array([[40233, 11433],
       [ 2669,  1645]], dtype=int64)

In [9]:
pca = PCA(n_components=5)
X_train_transformed = pca.fit_transform(X_train)
X_test_transformed = pca.transform(X_test)

In [11]:
# Use "hist" for constructing the trees, with early stopping enabled.
clf = SVC(gamma='scale', class_weight='balanced')
# Fit the model, test sets are used for early stopping.
clf.fit(X_train_transformed[:10000], y_train[:10000])
print(clf.score(X_test_transformed, y_test))

0.5485887817077527


In [12]:
y_pred = clf.predict(X_test_transformed)
confusion_matrix(y_test, y_pred)

array([[28662, 23004],
       [ 2266,  2048]], dtype=int64)